In [1]:
import numpy as np
import librosa as lb
import pandas as pd

from tqdm import tqdm
from sklearn.utils import shuffle
import os

In [2]:
train_path = './data/train'
test_path = './data/test'
feat_path = './features'

In [3]:
if not os.path.exists(feat_path):
    os.makedirs(feat_path)

In [4]:
GLOBAL_SEED = 42
NUM_CLASSES = 30
SR = 16000
LABELS = os.listdir(train_path)

## melspec feature

In [4]:
def get_melspec(x, sr, n_mels=256):
    melspec = lb.feature.melspectrogram(x, sr=sr, n_mels=n_mels, n_fft=2048, hop_length=512)
#     lb.power_to_db(melspec).astype(np.float32)
    return melspec


def crop_or_pad(y, length):
    if len(y) < length:
        y = np.concatenate([y, np.zeros(length - len(y))])
    elif len(y) > length:
        y = y[: length]
    return y

def preprocess_train(train_path):
    x, y = [], []
    for i, label in enumerate(LABELS):
        label_dir = os.path.join(train_path, label)
        for wav_file in tqdm(os.listdir(label_dir)):
            wav_path = os.path.join(train_path, label, wav_file)
            wav, sr = lb.load(wav_path, sr=SR)
            wav = crop_or_pad(wav, 1*SR)
            melspec = get_melspec(wav, sr)
            x.append(melspec)
            y.append(i)

    x, y = np.r_[x], np.r_[y]
#     x, y = shuffle(x, y, random_state=GLOBAL_SEED)
    return x.astype(np.float32), y.astype(np.int32)

def preprocess_test(test_path):
    x, keys = [], []

    for wav_file in tqdm(os.listdir(test_path)):
        wav_path = os.path.join(test_path, wav_file)
        wav, sr = lb.load(wav_path, sr=SR)
        wav = crop_or_pad(wav, 1*SR)
        melspec = get_melspec(wav, sr)
        x.append(melspec)
        keys.append(wav_file)
    x = np.r_[x]
    return x.astype(np.float32), keys

In [6]:
X_train, y_train = preprocess_train(train_path)

100%|██████████████████████████████████████████████████████████████████████████████| 2126/2126 [00:44<00:00, 47.55it/s]


In [8]:
X_train.shape

(57886, 256, 32)

In [19]:
X_test, test_names = preprocess_test(test_path)

100%|█████████████████████████████████████████████████████████████████████████████| 6835/6835 [00:35<00:00, 191.59it/s]


In [97]:
X_train.shape

(57886, 256, 32)

In [98]:
X_test.shape

(6835, 256, 32)

In [99]:
np.savez(os.path.join(feat_path, 'melspec_256'), X=np.concatenate([X_train, X_test], axis=0).transpose(0, 2, 1), y=y_train, test_names=test_names)

## Other features

In [5]:
import sklearn.preprocessing
import warnings
warnings.filterwarnings("ignore")

In [7]:
def get_melspec(x, sr, n_mels=256):
    melspec = lb.feature.melspectrogram(x, sr=sr, n_mels=n_mels, n_fft=2048, hop_length=512)
#     lb.power_to_db(melspec).astype(np.float32)
    return melspec

def get_audio_basic_feature(wav):
    d = lb.stft(wav, n_fft=2048)
    D = np.abs(d)**2
    S = lb.feature.melspectrogram(S=D, n_mels=128)
    mfcc = lb.feature.mfcc(S=lb.power_to_db(S), n_mfcc=40)
    norm_mfcc = sklearn.preprocessing.scale(mfcc, axis=1)
    zero_cross = lb.feature.zero_crossing_rate(wav, frame_length=2048)
    mag = lb.magphase(d)[0]
    centroid = lb.feature.spectral_centroid(S=mag)
    chromagram = lb.feature.chroma_stft(S=D, sr=SR)
    contrast = lb.feature.spectral_contrast(S=mag, sr=SR)
    bandwidth = lb.feature.spectral_bandwidth(S=mag, sr=SR)
    tonnetz = lb.feature.tonnetz(wav, sr=SR)
    concat = np.concatenate([norm_mfcc, zero_cross, centroid, chromagram, contrast, bandwidth, tonnetz])
#                 norm_concat = sklearn.preprocessing.scale(concat, axis=1)
    return concat


def crop_or_pad(y, length):
    if len(y) < length:
        y = np.concatenate([y, np.zeros(length - len(y))])
    elif len(y) > length:
        y = y[: length]
    return y

def preprocess_train(train_path):
    x, y = [], []
    for i, label in enumerate(LABELS):
        label_dir = os.path.join(train_path, label)
        for wav_file in tqdm(os.listdir(label_dir)):
            wav_path = os.path.join(train_path, label, wav_file)
            wav, sr = lb.load(wav_path, sr=SR)
            wav = crop_or_pad(wav, 1*SR)
            x.append(get_audio_basic_feature(wav))
    x, y = np.r_[x], np.r_[y]
#     x, y = shuffle(x, y, random_state=GLOBAL_SEED)
    return x.astype(np.float32), y.astype(np.int32)

def preprocess_test(test_path):
    x, keys = [], []
    for wav_file in tqdm(os.listdir(test_path)):
        wav_path = os.path.join(test_path, wav_file)
        wav, sr = lb.load(wav_path, sr=SR)
        wav = crop_or_pad(wav, 1*SR)
        x.append(get_audio_basic_feature(wav))
        keys.append(wav_file)
    x = np.r_[x]
    return x.astype(np.float32), keys

In [8]:
X_train, y_train = preprocess_train(train_path)

100%|██████████████████████████████████████████████████████████████████████████████| 2126/2126 [02:55<00:00, 12.09it/s]


In [9]:
X_test, test_names = preprocess_test(test_path)

100%|██████████████████████████████████████████████████████████████████████████████| 6835/6835 [09:13<00:00, 12.35it/s]


In [18]:
np.savez(os.path.join(feat_path, 'basic_feature'), X=np.concatenate([X_train, X_test], axis=0).transpose(0, 2, 1), y=y_train, test_names=test_names)

## To Image

In [5]:
SR = 65535

NUM_CLASSES = 30
LABELS = os.listdir(train_path)

In [6]:
def mono_to_color(
    X: np.ndarray, mean=None, std=None, norm_max=None, norm_min=None, eps=1e-6
):
    """
    Sources:
        https://www.kaggle.com/daisukelab/creating-fat2019-preprocessed-data
        https://www.kaggle.com/ttahara/training-birdsong-baseline-resnest50-fast
    """
    # Stack X as [X,X,X]
    X = np.stack([X, X, X], axis=-1)

    # Standardize
    mean = mean or X.mean()
    X = X - mean
    std = std or X.std()
    Xstd = X / (std + eps)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or _max
    norm_min = norm_min or _min
    if (_max - _min) > eps:
        # Normalize to [0, 255]
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V


def normalize(image, mean=None, std=None):
    image = image / 255.0
    if mean is not None and std is not None:
        image = (image - mean) / std
    return np.moveaxis(image, 2, 0).astype(np.float32)


def get_melspec(x, sr, n_mels=256):
    mel_spec = lb.feature.melspectrogram(x, sr=sr, n_mels=n_mels, n_fft=2048, hop_length=512, power=2)
    mel_spec = lb.power_to_db(mel_spec).astype(np.float32)
    image = mono_to_color(mel_spec)
#     image = normalize(image, mean=None, std=None)
    image = image.transpose(2, 1, 0)
#     image = resize(image, (224, 400)) 
    return image


def crop_or_pad(y, length):
    if len(y) < length:
        y = np.concatenate([y, np.zeros(length - len(y))])
    elif len(y) > length:
        y = y[: length]
    return y

def preprocess_train(train_path):
    x, y = [], []
    for i, label in enumerate(LABELS):
        label_dir = os.path.join(train_path, label)
        for wav_file in tqdm(os.listdir(label_dir)):
            wav_path = os.path.join(train_path, label, wav_file)
            wav, sr = lb.load(wav_path, sr=SR)
            wav = crop_or_pad(wav, 1*SR)
            melspec = get_melspec(wav, sr)
            x.append(melspec)
            y.append(i)

    x, y = np.r_[x], np.r_[y]
#     x, y = shuffle(x, y, random_state=GLOBAL_SEED)
    return x, y.astype(np.int32)

def preprocess_test(test_path):
    x, keys = [], []

    for wav_file in tqdm(os.listdir(test_path)):
        wav_path = os.path.join(test_path, wav_file)
        wav, sr = lb.load(wav_path, sr=SR)
        wav = crop_or_pad(wav, 1*SR)
        melspec = get_melspec(wav, sr)
        x.append(melspec)
        keys.append(wav_file)
    x = np.r_[x]
    return x, keys

In [7]:
X_train, y_train = preprocess_train(train_path)

100%|██████████████████████████████████████████████████████████████████████████████| 2126/2126 [11:55<00:00,  2.97it/s]


In [8]:
X_test, test_names = preprocess_test(test_path)

100%|██████████████████████████████████████████████████████████████████████████████| 6835/6835 [06:16<00:00, 18.16it/s]


In [9]:
np.savez(os.path.join(feat_path, 'image_128_256'), X=np.concatenate([X_train, X_test], axis=0), y=y_train, test_names=test_names)